<h1>Sexism Dataset in catalan</h1>


In [ ]:
!pip install -U sklearn
!pip install -U nltk
!pip install stopwordsiso

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
from pathlib import Path
import nltk
import re
from tqdm import tqdm
from nltk.corpus import stopwords
import pandas as pd
import stopwordsiso as stopwords
import snowballstemmer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn import svm
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import metrics
from sklearn.model_selection import cross_val_score

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Processing the data

In [ ]:
# processing the data
def load_data(filepath):
  data = pd.read_csv(filepath, sep='\t')
  X, y = [], []
  X = list(data['text'])
  y1 = list(data['task1'])
  y2 = list(data['task2'])

  return X, y1, y2

In [ ]:
print('Load data...')
Xtrain_, y1train, y2train = load_data('train.tsv')
Xtest_, y1test, y2test = load_data('test.tsv')

print('preprocess...')
stemmer = snowballstemmer.stemmer('catalan')

stop_words = stopwords.stopwords('ca')


def preprocess(text):
    text = text.lower()
    text = re.sub('@\S+', '', text) # Remove usernames
    text = re.sub('http[s]?://\S+', '', text) # Remove urls
    text  = re.sub('[\d\W_]+', ' ', text) # Remove numbers, emojis, simbols, ...
    text = text.split()
    text = [x for x in text if x not in stop_words]  # Eliminate the stopwords
    text = stemmer.stemWords(text) # Apply stemming
    return ' '.join(text)

Xtrain = []
for text in Xtrain_:
    Xtrain.append(preprocess(text))

print('Text: ', Xtrain[0])
print('Label 1:', y1train[0], 'Label 2:', y2train[0])

Xtest = []
for text in Xtest_:
    Xtest.append(preprocess(text))

print('Text: ', Xtest[0])
print('Label 1:', y1test[0], 'Label 2:', y2test[0])

Load data...
preprocess...
Text:  ella s autoanomena anti feminazi merda comentari vil ciutadà gran responsable tu sach much ghani bawri bawri hai bey
Label 1: sexist Label 2: ideological-inequality
Text:  representant l pennsilvània s horroritza política inicial movita johnson harrell ofès paraula jesús déu país principalment cristià resant déu jesús escàndol sarc coneguda estratègia mooslime jugar carta víctima demandes genoll s inclinarà
Label 1: non-sexist Label 2: non-sexist


In [ ]:
# Listing all stopwords used in this application
print(stop_words)

{'quin', 'ús', 'tal', 'alguns', 'igual', 'mons', 'vostres', 'més', 'vostra', 'contra', 'podeu', 'ací', 'i', 'saben', 'allò', 'apa', 'només', 'està', 'ser', 'teves', 'tot', 'aquelles', 'molts', 'fora', 'sons', 'així', 'estaven', 'em', 'amb', 'eres', 'vosaltres', 'sense', 'molt', 'inclòs', 'mateixes', 'cada', 'on', 'llarg', 'esteu', 'tota', 'una', 'aquí', 'havia', 'conseguir', 'són', 'potser', 'últim', 'els', 'feu', 'seus', 'semblants', 'semblant', 'qual', 'però', 'aleshores', 'dintre', 'jo', 'e', 'veu', 'son', 'molta', 'alguna', 'ta', 'seu', 'aquesta', 'allí', 'teniu', 'ah', 'puc', 'era', 'ambdues', 'mal', 'sou', 'llavors', 'cadascuna', 'a', 'ells', 'en', 'solament', 'quant', 'teus', 'unes', 'vaig', 'nostres', 'també', 'bé', 'et', 'què', 'quina', 'elles', 'altres', 'estic', 'malgrat', "s'han", 'éreu', 'després', 'estan', 'us', 'donat', 'la', "m'he", 'encara', 'meves', 'seves', 'poca', 'estat', 'nosaltres', 'som', 'oh', 'algun', 'poder', 'tampoc', "d'uns", 'tan', 'durant', 'sota', 'he', 

In [ ]:
# Showing the first 5 original and processed sentences
for i in range(5):
  print('Text original: ', Xtrain_[i])
  print('Text processed:', Xtrain[i])

Text original:  Ella s'autoanomena "anti-feminazi", què tal la merda amb el teu comentari vil sobre un ciutadà gran responsable tu sach much ghani bawri-bawri hai bey https://t.co/ZMxTDwsY5D
Text processed: ella s autoanomena anti feminazi merda comentari vil ciutadà gran responsable tu sach much ghani bawri bawri hai bey
Text original:  Ara, tornem a aquestes dones, les valentes i les belles, @Clare_Crawley i @tayshia. Aquestes nenes dolents es mereixen tant de crèdit per com ha anat aquesta temporada. Com a dona, he après molt d'ells i em sento més empoderada per esperar més en futures relacions.
Text processed: ara tornem dones valentes belles nenes dolents mereixen crèdit anat temporada dona après d sento empoderada esperar futures relacions
Text original:  @CurvyBandida @Xalynne_B Vaja, la teva faldilla és molt curta. Quina és la seva longitud? 5 polzades o més?
Text processed: vaja faldilla curta longitud polzades
Text original:  @AurelieGuiboud Increïble! Preciós! Però em vaig r

# Task 1

In [ ]:
# Adding a count vectorizer and feature selection to the following Pipeline

num_features=75
count_vectorizer = CountVectorizer()
feature_selector= SelectKBest(chi2, k=num_features)

text_clf = Pipeline([
    # add count vetorizer
    ('vec', count_vectorizer),
    # add feature selector
    ('fs', feature_selector),

    ('clf', MultinomialNB()),
])
text_clf.fit(Xtrain, y1train)

Pipeline(steps=[('vec', CountVectorizer()),
                ('fs',
                 SelectKBest(k=75,
                             score_func=<function chi2 at 0x7f958af46170>)),
                ('clf', MultinomialNB())])

In [ ]:
predicted = text_clf.predict(Xtest)

In [ ]:
text_clf2 = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB()),
])
text_clf2.fit(Xtrain, y1train)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', MultinomialNB())])

In [ ]:
predicted2 = text_clf2.predict(Xtest)

In [ ]:
text_clf3 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
text_clf3.fit(Xtrain, y1train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [ ]:
predicted3 = text_clf3.predict(Xtest)

# Evaluation

In [ ]:
# Evaluation
label_names = ['sexist', 'non-sexist']
print(metrics.classification_report(y1test, predicted,
     target_names=label_names))

print('Confusion matrix')
print(metrics.confusion_matrix(y1test, predicted))

              precision    recall  f1-score   support

      sexist       0.63      0.75      0.68      2087
  non-sexist       0.72      0.59      0.65      2281

    accuracy                           0.67      4368
   macro avg       0.68      0.67      0.67      4368
weighted avg       0.68      0.67      0.67      4368

Confusion matrix
[[1567  520]
 [ 926 1355]]


In [ ]:
label_names = ['sexist', 'non-sexist']
print(metrics.classification_report(y1test, predicted2,
     target_names=label_names))

print('Confusion matrix')
print(metrics.confusion_matrix(y1test, predicted2))

              precision    recall  f1-score   support

      sexist       0.67      0.65      0.66      2087
  non-sexist       0.69      0.71      0.70      2281

    accuracy                           0.68      4368
   macro avg       0.68      0.68      0.68      4368
weighted avg       0.68      0.68      0.68      4368

Confusion matrix
[[1360  727]
 [ 662 1619]]


In [ ]:
label_names = ['sexist', 'non-sexist']
print(metrics.classification_report(y1test, predicted3,
     target_names=label_names))

print('Confusion matrix')
print(metrics.confusion_matrix(y1test, predicted3))

              precision    recall  f1-score   support

      sexist       0.64      0.71      0.67      2087
  non-sexist       0.71      0.64      0.67      2281

    accuracy                           0.67      4368
   macro avg       0.67      0.67      0.67      4368
weighted avg       0.68      0.67      0.67      4368

Confusion matrix
[[1482  605]
 [ 828 1453]]


In [ ]:
# Cross-validation
scores = cross_val_score(text_clf, Xtrain+Xtest, y1train+y1test, cv=10, scoring='f1_macro', n_jobs=4)
print("scores: ", scores)
print("Avg score: ", scores.mean())
print("std", scores.std())

scores:  [0.68616602 0.67224441 0.67312674 0.64358738 0.64971044 0.64586769
 0.66296216 0.68604039 0.65321567 0.64817865]
Avg score:  0.6621099528377045
std 0.01552416619251664


In [ ]:
scores = cross_val_score(text_clf2, Xtrain+Xtest, y1train+y1test, cv=10, scoring='f1_macro', n_jobs=4)
print("scores: ", scores)
print("Avg score: ", scores.mean())
print("std", scores.std())

scores:  [0.69720902 0.7007529  0.68854991 0.70449387 0.69576223 0.65862354
 0.66645254 0.68035897 0.68413731 0.69367742]
Avg score:  0.6870017708169549
std 0.014152343682122908


In [ ]:
scores = cross_val_score(text_clf3, Xtrain+Xtest, y1train+y1test, cv=10, scoring='f1_macro', n_jobs=4)
print("scores: ", scores)
print("Avg score: ", scores.mean())
print("std", scores.std())

scores:  [0.69432821 0.70024564 0.69158309 0.71167878 0.70530389 0.6558913
 0.67740685 0.67994327 0.69664054 0.70427071]
Avg score:  0.6917292276120051
std 0.015720583822424167


# Task 2

In [ ]:
# Adding a count vectorizer and feature selection to the following Pipeline

num_features=80
count_vectorizer = CountVectorizer()
feature_selector= SelectKBest(chi2, k=num_features)

text_clf = Pipeline([
    # add count vetorizer
    ('vec', count_vectorizer),
    # add feature selector
    ('fs', feature_selector),

    ('clf', MultinomialNB()),
])
text_clf.fit(Xtrain, y2train)

Pipeline(steps=[('vec', CountVectorizer()),
                ('fs',
                 SelectKBest(k=80,
                             score_func=<function chi2 at 0x7f958af46170>)),
                ('clf', MultinomialNB())])

In [ ]:
predicted = text_clf.predict(Xtest)

In [ ]:
text_clf2 = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB()),
])
text_clf2.fit(Xtrain, y2train)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', MultinomialNB())])

In [ ]:
predicted2 = text_clf2.predict(Xtest)

In [ ]:
text_clf3 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
text_clf3.fit(Xtrain, y2train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [ ]:
predicted3 = text_clf3.predict(Xtest)

# Evaluation

In [ ]:
# Evaluation
label_names = ['ideological-inequality', 'stereotyping-dominance', 'objectification', 'sexual-violence', 'misogyny-non-sexual-violence', 'non-sexist']
print(metrics.classification_report(y2test, predicted,
     target_names=label_names))

print('Confusion matrix')
print(metrics.confusion_matrix(y2test, predicted))

                              precision    recall  f1-score   support

      ideological-inequality       0.81      0.34      0.48       621
      stereotyping-dominance       0.60      0.13      0.22       472
             objectification       0.55      0.93      0.69      2087
             sexual-violence       0.48      0.17      0.25       324
misogyny-non-sexual-violence       0.65      0.21      0.32       400
                  non-sexist       0.59      0.27      0.37       464

                    accuracy                           0.57      4368
                   macro avg       0.61      0.34      0.39      4368
                weighted avg       0.60      0.57      0.51      4368

Confusion matrix
[[ 210    2  396    1    3    9]
 [  11   62  373   11    6    9]
 [  29   20 1936   30   32   40]
 [   3   11  228   55    3   24]
 [   2    5  295    8   85    5]
 [   3    3  320   10    2  126]]


In [ ]:
label_names = ['ideological-inequality', 'stereotyping-dominance', 'objectification', 'sexual-violence', 'misogyny-non-sexual-violence', 'non-sexist']
print(metrics.classification_report(y2test, predicted2,
     target_names=label_names))

print('Confusion matrix')
print(metrics.confusion_matrix(y2test, predicted2))

                              precision    recall  f1-score   support

      ideological-inequality       0.64      0.27      0.38       621
      stereotyping-dominance       0.50      0.07      0.13       472
             objectification       0.52      0.94      0.67      2087
             sexual-violence       0.57      0.08      0.14       324
misogyny-non-sexual-violence       0.62      0.10      0.18       400
                  non-sexist       0.62      0.18      0.28       464

                    accuracy                           0.53      4368
                   macro avg       0.58      0.27      0.30      4368
                weighted avg       0.56      0.53      0.44      4368

Confusion matrix
[[ 169    1  445    1    1    4]
 [  21   35  409    1    2    4]
 [  40   21 1970   10   18   28]
 [   6    6  271   25    4   12]
 [   9    1  341    5   42    2]
 [  20    6  353    2    1   82]]


In [ ]:
label_names = ['ideological-inequality', 'stereotyping-dominance', 'objectification', 'sexual-violence', 'misogyny-non-sexual-violence', 'non-sexist']
print(metrics.classification_report(y2test, predicted3,
     target_names=label_names))

print('Confusion matrix')
print(metrics.confusion_matrix(y2test, predicted3))

                              precision    recall  f1-score   support

      ideological-inequality       0.88      0.01      0.02       621
      stereotyping-dominance       0.00      0.00      0.00       472
             objectification       0.48      1.00      0.65      2087
             sexual-violence       0.67      0.02      0.04       324
misogyny-non-sexual-violence       1.00      0.01      0.01       400
                  non-sexist       0.91      0.04      0.08       464

                    accuracy                           0.49      4368
                   macro avg       0.66      0.18      0.13      4368
                weighted avg       0.59      0.49      0.33      4368

Confusion matrix
[[   7    0  614    0    0    0]
 [   0    0  472    0    0    0]
 [   1    0 2085    1    0    0]
 [   0    0  316    6    0    2]
 [   0    0  398    0    2    0]
 [   0    1  441    2    0   20]]


In [ ]:
# Cross-validation
scores = cross_val_score(text_clf, Xtrain+Xtest, y2train+y2test, cv=10, scoring='f1_macro', n_jobs=4)
print("scores: ", scores)
print("Avg score: ", scores.mean())
print("std", scores.std())

scores:  [0.383817   0.36878206 0.39505584 0.41920285 0.38048265 0.41612508
 0.33876631 0.39141194 0.39657799 0.41862756]
Avg score:  0.3908849272013805
std 0.02371024826611616


In [ ]:
scores = cross_val_score(text_clf2, Xtrain+Xtest, y2train+y2test, cv=10, scoring='f1_macro', n_jobs=4)
print("scores: ", scores)
print("Avg score: ", scores.mean())
print("std", scores.std())

scores:  [0.33800685 0.33528131 0.37809949 0.40583497 0.37493114 0.37600934
 0.29311503 0.3236282  0.34023855 0.37219494]
Avg score:  0.3537339812484987
std 0.03153647954262611


In [ ]:
scores = cross_val_score(text_clf3, Xtrain+Xtest, y2train+y2test, cv=10, scoring='f1_macro', n_jobs=4)
print("scores: ", scores)
print("Avg score: ", scores.mean())
print("std", scores.std())

scores:  [0.15883266 0.14419306 0.14638832 0.17589983 0.15187291 0.14983047
 0.13276488 0.1415135  0.15341177 0.13904898]
Avg score:  0.1493756384019807
std 0.011383561876030046
